In [1]:
import os

import pandas as pd

In [9]:
SUPPLEMENTAL_DATA_DIR = os.path.expanduser("~/Desktop/Forny_2023_data")

SUPPLEMENTAL_DATA_FILES = {
    "transcriptomics" : {
        "file" : "42255_2022_720_MOESM3_ESM.xlsx",
        "sheet" : "Source Data transcriptomics"
    },
    "proteomics" : {
        "file" : "42255_2022_720_MOESM3_ESM.xlsx",
        "sheet" : "Source Data proteomics"
    },
    "phenotypes" : {
        "file" : "42255_2022_720_MOESM4_ESM.xlsx",
        "sheet" : 0
    }
}

In [10]:
# load
supplemental_data_path = {
    x : {
        "path" : os.path.join(SUPPLEMENTAL_DATA_DIR, y["file"]),
        "sheet" : y["sheet"]
    }
        for x, y in SUPPLEMENTAL_DATA_FILES.items()
}

assert all([os.path.isfile(x["path"]) for x in supplemental_data_path.values()])

supplemental_data = {
    x : pd.read_excel(y["path"], sheet_name = y["sheet"]) for x, y in supplemental_data_path.items()
}

In [13]:
# formatting
supplemental_data["transcriptomics"] = supplemental_data["transcriptomics"].rename({"Unnamed: 0" : "ensembl_gene"}, axis = 1).set_index("ensembl_gene")
supplemental_data["proteomics"] = supplemental_data["proteomics"].rename({"PG.ProteinAccessions" : "uniprot"}, axis = 1).set_index("uniprot")
supplemental_data["phenotypes"] = supplemental_data["phenotypes"].rename({"Unnamed: 0" : "patient_id"}, axis = 1).set_index("patient_id")

In [19]:
supplemental_data["transcriptomics"].head()

,MMA001,MMA002,MMA003,MMA004,MMA005,MMA006,MMA007,MMA008,MMA009,MMA010,...,MMA220,MMA222,MMA223,MMA224,MMA225,MMA226,MMA227,MMA228,MMA229,MMA230
ensembl_gene,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,1.873615,1.505270,0.986076,1.123403,5.806861,1.431328,1.251731,9.403244,1.353007,1.174570,...,12.565674,5.229523,6.640500,3.129463,1.731466,1.518375,1.121370,1.922590,3.458497,0.989769
ENSG00000188976,13.627592,7.954398,7.072510,7.722724,11.079538,6.171506,6.466709,12.337651,11.094306,5.950285,...,17.705565,9.782707,9.526287,7.936750,6.794779,8.044985,7.041855,8.283558,11.754802,7.057638
ENSG00000187961,1.037994,0.552903,0.436914,0.738782,1.691288,0.438173,0.606665,0.985185,1.486020,0.152516,...,0.839992,0.810122,0.458880,0.343265,0.510889,0.270509,0.218708,0.439597,0.429895,0.604521
ENSG00000187583,0.698824,0.041128,0.024047,0.061751,0.238684,0.037082,0.019218,0.483886,0.058053,0.030468,...,0.862213,0.126554,0.429522,0.041436,0.042376,0.000000,0.076661,0.000000,0.012485,0.014054
ENSG00000187642,0.166474,0.403885,0.033554,0.083613,0.069547,0.042103,0.041908,0.117374,0.019715,0.007449,...,0.185598,0.027146,0.098651,0.000000,0.026749,0.000000,0.011748,0.000000,0.000000,0.023950


In [20]:
supplemental_data["proteomics"].head()

,PG.ProteinDescriptions,PG.ProteinNames,PG.Qvalue,MMA001,MMA002,MMA003,MMA004,MMA005,MMA006,MMA007,...,MMA221,MMA222,MMA223,MMA224,MMA225,MMA226,MMA227,MMA228,MMA229,MMA230
uniprot,,,,,,,,,,,,,,,,,,,,,
A0AVF1,Intraflagellar transport protein 56,IFT56_HUMAN,0.005996,83727.578125,119782.273438,435030.031250,351463.812500,473821.687500,462218.531250,461786.062500,...,96052.937500,203895.218750,367420.562500,128790.132812,258611.218750,236174.890625,281066.500000,57388.179688,21557.371094,165707.18750
A0AVT1,Ubiquitin-like modifier-activating enzyme 6,UBA6_HUMAN,0.000085,37115.953125,46101.957031,65014.164062,71438.554688,42789.007812,41797.777344,40118.589844,...,42814.156250,49667.570312,36617.558594,63397.425781,51176.625000,42973.957031,55355.019531,33448.113281,42359.183594,45239.43750
A0FGR8,Extended synaptotagmin-2,ESYT2_HUMAN,0.000017,40237.117188,217922.515625,197898.968750,122243.023438,152602.468750,170386.234375,158019.125000,...,160844.015625,131429.296875,147733.250000,130756.320312,181981.031250,110166.304688,131816.656250,139880.625000,199221.109375,108320.78125
A1AG_BOVINAlpha-1-acidglycoproteinOS=BostaurusGN=ORM1PE=2SV=1;CONT_Q3SZR3,;(Bos taurus) Alpha-1-acid glycoprotein precursor,NaN,0.000001,46269.468750,456155.343750,76156.734375,70375.281250,71832.406250,88579.406250,80614.843750,...,199413.093750,137026.562500,268573.125000,290260.312500,282464.937500,443858.531250,770606.937500,227444.343750,495527.593750,439083.68750
A1L0T0,Acetolactate synthase-like protein,ILVBL_HUMAN,0.000017,55125.847656,63051.160156,31387.976562,40272.636719,49587.910156,39840.132812,37784.460938,...,57427.914062,77553.562500,42669.714844,61045.855469,34615.488281,37323.367188,65609.390625,62056.046875,52899.207031,36870.06250


In [21]:
supplemental_data["phenotypes"].head()

,case,gender,n_passage,date_collection,date_freezing,consanguinity,onset_age,OHCblMinus,OHCblPlus,ratio,...,cssModif,irritability,hyperventilation,hypothermia,somnolence,vomiting,dehydration,feeding_problem,GFR,responsive_to_acute_treatment
patient_id,,,,,,,,,,,,,,,,,,,,,
MMA001,1,1,3.0,2001,2001,0,28.0,256.0,468.0,1.83,...,1,0,0,0,0,0,0,0,NaN,1
MMA002,1,1,3.0,2001,2001,1,1.0,1610.0,1544.0,0.96,...,2,0,0,0,0,0,1,1,NaN,0
MMA003,1,1,7.0,1991,1996,1,3.0,101.0,76.0,0.76,...,2,0,1,1,0,0,1,0,NaN,0
MMA004,1,0,5.0,1995,1996,0,3.0,220.0,219.0,1.00,...,2,0,0,0,0,0,0,0,NaN,1
MMA005,1,0,3.0,1995,1996,0,1.0,1830.0,1968.0,1.08,...,1,0,0,0,0,0,0,0,NaN,0


In [22]:
supplemental_data["phenotypes"].value_counts("case")

case
1    210
0     20
dtype: int64